<a href="https://colab.research.google.com/github/gab-palmeri/aml-geolocalization/blob/develop/step_4_reranking_geowarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to train the sf-xs dataset with GeoWarp. The resulting model can be then used for reranking.

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [1]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"
!pip3 install "kornia==0.6.2"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 29.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 10.1 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=26bbac307dc6030737e9c972702b2e3274ce7f211f0b3b0ea5e9f594d6616a59
  Stored in directory: /root/.cache/pip/wheels/65/e2/d8/878a8cc986641056fbfebefc4d8eb64238a7b6d3426e86b447
Successfully built utm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 31.8 MB/s 
cuda:0


# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [1]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )

use_mount = True
resume_model = True

if use_mount:
  drive.mount('/content/drive')

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")


# pretrained baselines for geowarp
if not os.path.isdir("/content/pretrained_baselines/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/project6/geowarp/pretrained_baselines.zip"

if not os.path.isdir("/content/pretrained_baselines/"):
  raise FileNotFoundError(f"Can't download pretrained_baselines for geowarp")


Output streaming troncato alle ultime 5000 righe.
 inflated: small/val/queries/@0548208.92@4184407.49@10@S@037.80578@-122.45235@OeX66vkQRPXMFrotCfkGCg@@360@@@@201508@@.jpg
 inflated: small/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@60@@@@201402@@.jpg
 inflated: small/val/queries/@0548208.92@4184407.49@10@S@037.80578@-122.45235@OeX66vkQRPXMFrotCfkGCg@@60@@@@201508@@.jpg
 inflated: small/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@90@@@@201402@@.jpg
 inflated: small/val/queries/@0548208.92@4184407.49@10@S@037.80578@-122.45235@OeX66vkQRPXMFrotCfkGCg@@90@@@@201508@@.jpg
 inflated: small/val/queries/@0548258.61@4173657.59@10@S@037.70889@-122.45250@5ktSqfqc-u2QOs68lw9sDw@@120@@@@201409@@.jpg
 inflated: small/val/queries/@0548258.48@4179554.69@10@S@037.76204@-122.45211@GGArRnpN7_AJXIx-El-9XQ@@0@@@@200711@@.jpg
 inflated: small/val/queries/@0548258.61@4173657.59@10@S@037.70889@-122.45250@5ktSqfqc-u2QOs68lw9sDw@@

# Download Code

Clone of the repository of our Team

In [2]:
# download our code
if not os.path.isdir("/content/Team"):
    !git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
    !mv "/content/aml-geolocalization/" "/content/Team"

Cloning into 'CosPlace'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 174 (delta 61), reused 44 (delta 44), pack-reused 106
Receiving objects: 100% (174/174), 45.67 KiB | 11.42 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Cloning into 'geo_warp'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 98 (delta 44), reused 47 (delta 9), pack-reused 0
Unpacking objects: 100% (98/98), done.
Cloning into 'aml-geolocalization'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 174 (delta 35), reused 40 (delta 30), pack-reused 125
Receiving objects: 100% (174/174), 2.02 MiB | 29.54 MiB/s, done.
Resolving deltas: 100% (66/66), done.


# Training GeoWarp

### Choose Backbone for Training

In [ ]:
backbone = "alexnet"

In [ ]:
backbone = "resnet50"

In [ ]:
backbone = "vgg16"

### Choose pooling method

In [ ]:
pooling = "gem"

In [ ]:
pooling = "netvlad"

### Choose the number of epochs

In [ ]:
n_epochs = 10 # change it with your desired value

### Training

In [12]:
!python3 /content/Team/GeoWarp/train.py --arch backbone --pooling pooling --resume_fe /content/$backbone_$pooling.pth --n_epochs n_epochs

Output streaming troncato alle ultime 5000 righe.
            4.2014e-01,  2.1813e-01],
          [-5.0096e-01, -5.0375e-01, -5.2031e-01,  ...,  4.7143e-01,
            3.5565e-01,  7.2584e-02]],

         [[ 1.5896e+00,  1.5435e+00,  1.4946e+00,  ...,  1.1585e+00,
            1.1584e+00,  1.1583e+00],
          [ 1.6057e+00,  1.5500e+00,  1.4893e+00,  ...,  1.1585e+00,
            1.1585e+00,  1.1585e+00],
          [ 1.6172e+00,  1.5589e+00,  1.4932e+00,  ...,  1.1557e+00,
            1.1565e+00,  1.1574e+00],
          ...,
          [-1.1744e+00, -1.1970e+00, -1.2187e+00,  ..., -6.5188e-01,
           -6.0195e-01, -7.0585e-01],
          [-1.2482e+00, -1.2662e+00, -1.2884e+00,  ..., -6.0160e-01,
           -6.1978e-01, -8.3276e-01],
          [-1.3334e+00, -1.3641e+00, -1.4013e+00,  ..., -5.5537e-01,
           -6.8064e-01, -9.6505e-01]]]])
2022-12-29 20:54:29   warped_images
2022-12-29 20:54:29   rectangle_points
2022-12-29 20:54:29   tensor([[[-1., -1.],
         [ 1., -1.],
    

# Save results

To save the results, just download the file at /content/runs/default/time/checkpoints/homography_regression.pth (the last epoch).